# Data Collection
Getting data from 2012 to 2016  
Years Prior to 2012 Currently have a bug when specifying NAICS#### as one of the columns
Years Prior to 2012 also only have the totals for all sectors not each sector.  
Years after 2017 not available  
Zip level not available -> https://api.census.gov/data/2011/cbp/variables.html  
ended up not using 2017 since the NAICS 2017 wasn't available in master_crrosswalk

In [1]:
import csv
import requests as r
import pandas as pd
import zipfile, io
import os
from tqdm import tqdm
import pathlib
import pandas as pd
api_headers = {}
api_headers['x-api-key'] = '975f39a54e48438ceebf303d6018e34db212e804'

In [2]:
# Set a relative location to save the data from the request
repo_dir = pathlib.Path().cwd()
raw_data_dir = repo_dir / 'data_raw'
out_data_dir = raw_data_dir / 'BEA_Industry_Factors'

# Create new folders if needed
#state_data_dir = out_data_dir / 'state_level'
#if not state_data_dir.exists():
#    state_data_dir.mkdir(parents=True)
    
county_data_dir = out_data_dir / 'county_level'
if not county_data_dir.exists():
    county_data_dir.mkdir(parents=True)
    
#zip_data_dir = out_data_dir / 'zip_level'
#if not county_data_dir.exists():
#    county_data_dir.mkdir(parents=True)

In [3]:
county_data_dir

WindowsPath('C:/Users/Owner/Documents/modelearth/community/info/data/data_raw/BEA_Industry_Factors/county_level')

In [4]:
# Load the state FIPS codes key
state_fips = pd.read_csv(out_data_dir / 'state_fips.csv', usecols=['Name', 'Postal Code', 'FIPS'])
state_fips = state_fips.head(50)  # <-- limit to only US states, not teritories

In [5]:
# Base URL for the API call
base_url = "https://api.census.gov/data"

#
# NOTE Years Prior to 2012 Currently have a bug when specifying NAICS#### as one of the columns
#      - stick to 2012 and later for now
#


def get_county_cbp(fips, state, years):
    count=0
    for year in years:
        print(f"Getting data for state: {state}\tyear: {year}")
        if year>=2000 and year<=2002:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS1997_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year>=2003 and year<=2007:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2002_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year>=2008 and year<=2011:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2007_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year>=2012 and year<=2016:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year==2017:
            columns_to_select = "GEO_ID,NAME,COUNTY,YEAR,NAICS2017,NAICS2017_LABEL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
    
    
        response = r.get(url, headers=api_headers)

        with open(county_data_dir / f"industriesPerCounty_{str.lower(state.replace(' ', ''))}_{year}.csv",'w') as resultPath:
            for line in response.text.strip().split('\n'):
                line=line.replace('[',"").replace(']',"")
                resultPath.write(line + "\n")

        print("  > Finished CSV for year"+str(year))

In [152]:
for fips in state_fips.FIPS.unique():
     state = state_fips.query(f'FIPS=={fips}').values[0][0]
     #years=range(2012,2018)
     years=range(2012,2018)
     get_county_cbp(fips, state, years)

Getting data for state: Alabama	year: 2008
  > Finished CSV for year2008
Getting data for state: Alabama	year: 2009
  > Finished CSV for year2009
Getting data for state: Alabama	year: 2010
  > Finished CSV for year2010
Getting data for state: Alabama	year: 2011
  > Finished CSV for year2011
Getting data for state: Alaska	year: 2008
  > Finished CSV for year2008
Getting data for state: Alaska	year: 2009
  > Finished CSV for year2009
Getting data for state: Alaska	year: 2010
  > Finished CSV for year2010
Getting data for state: Alaska	year: 2011
  > Finished CSV for year2011
Getting data for state: Arizona	year: 2008
  > Finished CSV for year2008
Getting data for state: Arizona	year: 2009
  > Finished CSV for year2009
Getting data for state: Arizona	year: 2010
  > Finished CSV for year2010
Getting data for state: Arizona	year: 2011
  > Finished CSV for year2011
Getting data for state: Arkansas	year: 2008
  > Finished CSV for year2008
Getting data for state: Arkansas	year: 2009
  > Finish

KeyboardInterrupt: 

# Data Aggregation
This part allows us to manage different Fips level (county/state) and different NAICS level (sector/industry/etc...)  
Trick: Getting the NAICS code from all the NAICS2012 files that we downloaded

In [6]:
# Load the 2012 data

def load_all_states2012(bea_data_dir):
    
    for i in range(2012,2017):
        #print(i)
        x="_"+str(i)
        files = [f for f in bea_data_dir.iterdir() if x in f.name]

        for f in files:
            #print(f)

            df = pd.read_csv(f,encoding='latin-1',dtype={"NAICS2012": str}).drop("Unnamed: 11", axis=1)
            #df = pd.read_csv(f)
            #df[df['NAICS2012'].str.len() != 5]
            #b=df[df['NAICS2012'].str.len() == 5]
            df['is5'] = df['NAICS2012'].apply(lambda x: 'True' if len(x) == 5 else 'False')
                #print("hh"+df.NAICS2012)
                #print("hghg"+len(df.NAICS2012))
            #df['NAICS_Sector'] = df['NAICS2012'].apply(lambda v: v[:2] if df['is5']=='False' else 1).apply(int)
            df.loc[(df['is5'] == 'True') & (df['NAICS2012'].apply(lambda v: v[2:3]) == '-'), 'NAICS_Sector'] = df['NAICS2012']
            df.loc[(df['is5'] == 'True') & (df['NAICS2012'].apply(lambda v: v[2:3]) != '-'), 'NAICS_Sector'] = df['NAICS2012'].apply(lambda v: v[:2])
            df.loc[(df['is5'] == 'False') , 'NAICS_Sector'] = df['NAICS2012'].apply(lambda v: v[:2])
                #print("hgh"+df.NAICS2012[1:2])
                #if (df.NAICS2012.apply(lambda v: v[0:1]).apply(int)==1).bool():
                #    print("bye")

            yield df
    
df = pd.concat(load_all_states2012(county_data_dir)).drop("is5", axis=1)

# test = list(load_all_states(bea_data_dir))

# df

In [7]:
df=df.drop("county", axis=1)
df

,GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN,state,NAICS_Sector
0,0500000US01001,"Autauga County, Alabama",1,2012,00,Total for all sectors,817,9908,280188,1,00
1,0500000US01001,"Autauga County, Alabama",1,2012,11,"Agriculture, forestry, fishing and hunting",6,30,1102,1,11
2,0500000US01001,"Autauga County, Alabama",1,2012,113,Forestry and logging,6,30,1102,1,11
3,0500000US01001,"Autauga County, Alabama",1,2012,1133,Logging,6,30,1102,1,11
4,0500000US01001,"Autauga County, Alabama",1,2012,2211,"Electric power generation, transmission and di...",8,182,16657,1,22
...,...,...,...,...,...,...,...,...,...,...,...
15091,04000S0US56,Wyoming -- Statewide,999,2016,621999,All other miscellaneous ambulatory health care...,1,0,0,56,62
15092,04000S0US56,Wyoming -- Statewide,999,2016,81,Other services (except public administration),1,0,0,56,81
15093,04000S0US56,Wyoming -- Statewide,999,2016,812,Personal and laundry services,1,0,0,56,81
15094,04000S0US56,Wyoming -- Statewide,999,2016,81299,All other personal services,1,0,0,56,81


### Process FIPS Code
FIPS is the federal/census unique ID for each geographic area.  States have 2 digives and counties have 5

In [10]:
# Process FIPS code
df['fips'] = df.GEO_ID.apply(lambda GID: GID.split('US')[1])
# # Remove the state_level data
# df_state = df[df['fips'].str.len() == 2]
# df_county = df[df['fips'].str.len() != 2]

def county_level(df):
    return df[df['id'].str.len() == 5]

def state_level(df):
    return df[df['id'].str.len() == 2]

### Group all data per sector for  every year and average them  
Does it make sense to sum up instead of average?!

In [20]:
newDF=df.groupby(['fips','state','COUNTY','GEO_TTL','NAICS2012','NAICS2012_TTL','NAICS_Sector'],as_index=False).mean()
newDF=newDF.drop("YEAR", axis=1)

In [21]:
#newDF=newDF.astype({'fips': 'int'})
newDF=newDF.rename(columns={"fips": "id", "NAICS2012": "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
#b=b[["id","relevant_naics","emp","payann","estab"]]
newDF

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
0,01,1,999,Alabama -- Statewide,00,Total for all sectors,00,405.80,42373.4,1333508.6
1,01,1,999,Alabama -- Statewide,21,"Mining, quarrying, and oil and gas extraction",21,18.00,386.4,27332.2
2,01,1,999,Alabama -- Statewide,211,Oil and gas extraction,21,4.20,31.6,3418.2
3,01,1,999,Alabama -- Statewide,2111,Oil and gas extraction,21,4.20,31.6,3418.2
4,01,1,999,Alabama -- Statewide,21111,Oil and gas extraction,21,4.20,31.6,3418.2
...,...,...,...,...,...,...,...,...,...,...
2365246,56045,56,45,"Weston County, Wyoming",81391,Business associations,81,1.00,0.0,0.0
2365247,56045,56,45,"Weston County, Wyoming",813910,Business associations,81,1.00,0.0,0.0
2365248,56045,56,45,"Weston County, Wyoming",81393,Labor unions and similar labor organizations,81,1.00,0.0,0.0
2365249,56045,56,45,"Weston County, Wyoming",813930,Labor unions and similar labor organizations,81,1.00,0.0,0.0


In [71]:
newDF.head(50)

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
0,01,1,999,Alabama -- Statewide,00,Total for all sectors,00,405.800000,42373.4,1333508.6
1,01,1,999,Alabama -- Statewide,21,"Mining, quarrying, and oil and gas extraction",21,18.000000,386.4,27332.2
2,01,1,999,Alabama -- Statewide,211,Oil and gas extraction,21,4.200000,31.6,3418.2
3,01,1,999,Alabama -- Statewide,2111,Oil and gas extraction,21,4.200000,31.6,3418.2
4,01,1,999,Alabama -- Statewide,21111,Oil and gas extraction,21,4.200000,31.6,3418.2
5,01,1,999,Alabama -- Statewide,211111,Crude petroleum and natural gas extraction,21,4.200000,31.6,3418.2
6,01,1,999,Alabama -- Statewide,213,Support activities for mining,21,13.800000,236.6,13282.6
7,01,1,999,Alabama -- Statewide,2131,Support activities for mining,21,13.800000,236.6,13282.6
8,01,1,999,Alabama -- Statewide,21311,Support activities for mining,21,13.800000,236.6,13282.6
9,01,1,999,Alabama -- Statewide,213111,Drilling oil and gas wells,21,5.800000,48.4,1924.4


### Group data by NAICS Sector

NAICS is the North American Industry Classification System. The coarsest level of classification is the *Sector*.

The organization of NAICS is as follows:  <-- from [this page](https://www.census.gov/programs-surveys/economic-census/guidance/understanding-naics.html) on census.gov
- Sector: 2-digit code
    - Subsector: 3-digit code
        - Industry Group: 4-digit code
            - NAICS Industry: 5-digit code
                - National Industry: 6-digit code

Start by grouping the data by sector:

In [22]:
def naics_level(df, naics_level):
    return df[df['relevant_naics'].str.len() == naics_level]

In [23]:
#df.head()
# df_county['NAICS2012'].apply(lambda v: v.split('-')[0])

df_naics_2 = naics_level(newDF, 2).reset_index(drop=True)
df_naics_3 = naics_level(newDF, 3).reset_index(drop=True)
df_naics_4 = naics_level(newDF, 4).reset_index(drop=True)
df_naics_5 = naics_level(newDF, 5).reset_index(drop=True)
df_naics_6 = naics_level(newDF, 6).reset_index(drop=True)

# First remove all the per-county totals across all sectors
#df_totals = df_naics_2[df_naics_2.NAICS2012 == '00']
df_naics_2 = df_naics_2[df_naics_2.relevant_naics != '00']
df_naics_3 = df_naics_3[df_naics_3.relevant_naics != '00']
df_naics_4 = df_naics_4[df_naics_4.relevant_naics != '00']
df_naics_5 = df_naics_5[df_naics_5.relevant_naics != '00']
df_naics_6 = df_naics_6[df_naics_6.relevant_naics != '00']

In [62]:
s2=state_level(df_naics_2)
c2=county_level(df_naics_2)
s3=state_level(df_naics_3)
c3=county_level(df_naics_3)
s4=state_level(df_naics_4)
c4=county_level(df_naics_4)
s5=state_level(df_naics_5)
c5=county_level(df_naics_5)
s6=state_level(df_naics_6)
c6=county_level(df_naics_6)

In [63]:
s2.to_csv('s2.tsv',sep='\t')
c2.to_csv('c2.tsv',sep='\t')
s3.to_csv('s3.tsv',sep='\t')
c3.to_csv('c3.tsv',sep='\t')
s4.to_csv('s4.tsv',sep='\t')
c4.to_csv('c4.tsv',sep='\t')
s5.to_csv('s5.tsv',sep='\t')
c5.to_csv('c5.tsv',sep='\t')
s6.to_csv('s6.tsv',sep='\t')
c6.to_csv('c6.tsv',sep='\t')

In [1]:
c5

NameError: name 'c5' is not defined

In [66]:
s2.state.unique()

array([ 1,  2,  4,  5,  6,  8,  9, 10, 12, 13, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56],
      dtype=int64)

In [338]:
newDF

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
0,01,1,999,Alabama -- Statewide,00,Total for all sectors,00,405.80,42373.4,1333508.6
1,01,1,999,Alabama -- Statewide,21,"Mining, quarrying, and oil and gas extraction",21,18.00,386.4,27332.2
2,01,1,999,Alabama -- Statewide,211,Oil and gas extraction,21,4.20,31.6,3418.2
3,01,1,999,Alabama -- Statewide,2111,Oil and gas extraction,21,4.20,31.6,3418.2
4,01,1,999,Alabama -- Statewide,21111,Oil and gas extraction,21,4.20,31.6,3418.2
...,...,...,...,...,...,...,...,...,...,...
2365246,56045,56,45,"Weston County, Wyoming",81391,Business associations,81,1.00,0.0,0.0
2365247,56045,56,45,"Weston County, Wyoming",813910,Business associations,81,1.00,0.0,0.0
2365248,56045,56,45,"Weston County, Wyoming",81393,Labor unions and similar labor organizations,81,1.00,0.0,0.0
2365249,56045,56,45,"Weston County, Wyoming",813930,Labor unions and similar labor organizations,81,1.00,0.0,0.0


In [340]:
newDF.to_csv('all.tsv',sep='\t')

# The statewide data does not include NAICS starting with 1!

In [74]:
s3.head(50)

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
0,01,1,999,Alabama -- Statewide,211,Oil and gas extraction,21,4.200000,31.6,3418.2
1,01,1,999,Alabama -- Statewide,213,Support activities for mining,21,13.800000,236.6,13282.6
2,01,1,999,Alabama -- Statewide,221,Utilities,22,1.800000,0.0,0.0
3,01,1,999,Alabama -- Statewide,236,Construction of buildings,23,1.000000,0.0,0.0
4,01,1,999,Alabama -- Statewide,237,Heavy and civil engineering construction,23,3.400000,307.2,15727.2
5,01,1,999,Alabama -- Statewide,238,Specialty trade contractors,23,1.333333,0.0,0.0
6,01,1,999,Alabama -- Statewide,423,"Merchant wholesalers, durable goods",42,59.000000,581.8,61443.6
7,01,1,999,Alabama -- Statewide,424,"Merchant wholesalers, nondurable goods",42,43.000000,600.2,62138.8
8,01,1,999,Alabama -- Statewide,425,Wholesale electronic markets and agents and br...,42,5.200000,119.4,1962.4
9,01,1,999,Alabama -- Statewide,443,Electronics and appliance stores,44,1.000000,0.0,0.0


In [24]:
state_level(df_naics_2)

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
1,01,1,999,Alabama -- Statewide,21,"Mining, quarrying, and oil and gas extraction",21,18.0,386.40,27332.20
2,01,1,999,Alabama -- Statewide,22,Utilities,22,1.8,0.00,0.00
3,01,1,999,Alabama -- Statewide,23,Construction,23,5.0,449.60,24474.20
4,01,1,999,Alabama -- Statewide,42,Wholesale trade,42,107.2,1480.00,128801.00
5,01,1,999,Alabama -- Statewide,51,Information,51,27.8,202.40,17455.40
...,...,...,...,...,...,...,...,...,...,...
53590,56,56,999,Wyoming -- Statewide,55,Management of companies and enterprises,55,1.8,0.00,0.00
53591,56,56,999,Wyoming -- Statewide,56,Administrative and support and waste managemen...,56,15.8,66.60,2448.40
53592,56,56,999,Wyoming -- Statewide,61,Educational services,61,2.5,9.25,83.75
53593,56,56,999,Wyoming -- Statewide,62,Health care and social assistance,62,4.2,20.20,1023.80


In [25]:
a=county_level(df_naics_2)

In [26]:
county_level(df_naics_6)

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
156,01001,1,1,"Autauga County, Alabama",113310,Logging,11,6.0,54.8,2611.2
157,01001,1,1,"Autauga County, Alabama",115112,"Soil preparation, planting, and cultivating",11,1.0,0.0,0.0
158,01001,1,1,"Autauga County, Alabama",115310,Support activities for forestry,11,1.0,0.0,0.0
159,01001,1,1,"Autauga County, Alabama",212319,Other crushed and broken stone mining and quar...,21,1.0,0.0,0.0
160,01001,1,1,"Autauga County, Alabama",212321,Construction sand and gravel mining,21,1.6,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
892472,56045,56,45,"Weston County, Wyoming",813110,Religious organizations,81,8.6,10.6,226.8
892473,56045,56,45,"Weston County, Wyoming",813312,"Environment, conservation and wildlife organiz...",81,1.0,0.0,0.0
892474,56045,56,45,"Weston County, Wyoming",813319,Other social advocacy organizations,81,1.0,0.0,0.0
892475,56045,56,45,"Weston County, Wyoming",813910,Business associations,81,1.0,0.0,0.0


In [28]:
df_naics_5[df_naics_5.id=='13001']

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
109444,13001,13,1,"Appling County, Georgia",11331,Logging,11,10.2,34.6,1709.8
109445,13001,13,1,"Appling County, Georgia",11511,Support activities for crop production,11,1.0,0.0,0.0
109446,13001,13,1,"Appling County, Georgia",22111,Electric power generation,22,1.8,0.0,0.0
109447,13001,13,1,"Appling County, Georgia",22112,"Electric power transmission, control, and dist...",22,5.2,0.0,0.0
109448,13001,13,1,"Appling County, Georgia",23611,Residential building construction,23,2.8,5.2,137.6
...,...,...,...,...,...,...,...,...,...,...
109609,13001,13,1,"Appling County, Georgia",81299,All other personal services,81,1.4,0.0,0.0
109610,13001,13,1,"Appling County, Georgia",81311,Religious organizations,81,21.6,85.6,1546.8
109611,13001,13,1,"Appling County, Georgia",81331,Social advocacy organizations,81,1.2,0.0,0.0
109612,13001,13,1,"Appling County, Georgia",81341,Civic and social organizations,81,1.4,0.0,0.0


In [225]:
#b=a.astype({'fips': 'int','NAICS2012': 'int'})
#b.dtypes

fips               int32
state              int64
COUNTY             int64
GEO_TTL           object
NAICS2012          int32
NAICS2012_TTL     object
NAICS_Sector      object
ESTAB            float64
EMP              float64
PAYANN           float64
dtype: object

In [29]:
#a.to_csv('a1.tsv',sep='\t')

In [30]:
#b=a.astype({'fips': 'int','NAICS2012': 'int'})
#b=b.rename(columns={"fips": "id", "NAICS2012": "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
#b=b[["id","relevant_naics","emp","payann","estab"]]
#b

# NAICS code to name translation

In [55]:
NAICS_codes = pd.read_csv(out_data_dir / 'Crosswalk_MasterCrosswalk.csv', usecols=['2012_NAICS_Code', '2012_NAICS_Industry'])

In [56]:
NAICS_codes=NAICS_codes.rename(columns={"2012_NAICS_Code": "relevant_naics", "2012_NAICS_Industry": "industry_detail"})

In [57]:
NAICS_codes=NAICS_codes.dropna()

In [58]:
NAICS_codes=NAICS_codes.drop_duplicates()

In [59]:
NAICS_codes

,relevant_naics,industry_detail
0,11.0,"Agriculture, Forestry, Fishing and Hunting"
13,111.0,Crop Production
19,1111.0,Oilseed and Grain Farming
21,11111.0,Soybean Farming
22,111110.0,Soybean Farming
...,...,...
3885,9281.0,National Security and International Affairs
3887,92811.0,National Security
3888,928110.0,National Security
3889,92812.0,International Affairs


In [61]:
NAICS_codes.to_csv('industry_ID_list.csv')

# Making a states json

In [122]:
stateFips = pd.read_csv(out_data_dir / 'state_fips.csv')

In [123]:
stateFips=stateFips.drop(['Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'],axis=1)

In [112]:
#stateFips=stateFips.set_index('Name')

In [124]:
stateFips

,Name,Postal Code,FIPS
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6
5,Colorado,CO,8
6,Connecticut,CT,9
7,Delaware,DE,10
8,Florida,FL,12
9,Georgia,GA,13


In [127]:
stateFips.to_json('states.json', orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)

# Making county to fips csv

In [321]:
countyDF=c2[['GEO_TTL','id']].drop_duplicates()

In [322]:
countyDF

,GEO_TTL,id
16,"Autauga County, Alabama",01001
34,"Baldwin County, Alabama",01003
52,"Barbour County, Alabama",01005
70,"Bibb County, Alabama",01007
88,"Blount County, Alabama",01009
...,...,...
53916,"Sweetwater County, Wyoming",56037
53934,"Teton County, Wyoming",56039
53952,"Uinta County, Wyoming",56041
53969,"Washakie County, Wyoming",56043


In [323]:
countyDF['hascomma'] =countyDF['GEO_TTL'].apply(lambda x: 'True' if ',' in x else 'False')
countyDF
#countyDF['county'] = df.GEO_TTL.apply(lambda GTT: GTT.split(',')[0])

,GEO_TTL,id,hascomma
16,"Autauga County, Alabama",01001,True
34,"Baldwin County, Alabama",01003,True
52,"Barbour County, Alabama",01005,True
70,"Bibb County, Alabama",01007,True
88,"Blount County, Alabama",01009,True
...,...,...,...
53916,"Sweetwater County, Wyoming",56037,True
53934,"Teton County, Wyoming",56039,True
53952,"Uinta County, Wyoming",56041,True
53969,"Washakie County, Wyoming",56043,True


In [324]:
countyDF.loc[(countyDF['hascomma'] == 'True'), 'county'] = countyDF.GEO_TTL.apply(lambda GTT: GTT.split(', ')[0])
countyDF.loc[(countyDF['hascomma'] == 'True'), 'state'] = countyDF.GEO_TTL.apply(lambda GTT: GTT.split(', ')[1])

In [325]:
countyDF=countyDF[['state','county','id']].drop_duplicates()

In [326]:
countyDF

,state,county,id
16,Alabama,Autauga County,01001
34,Alabama,Baldwin County,01003
52,Alabama,Barbour County,01005
70,Alabama,Bibb County,01007
88,Alabama,Blount County,01009
...,...,...,...
53916,Wyoming,Sweetwater County,56037
53934,Wyoming,Teton County,56039
53952,Wyoming,Uinta County,56041
53969,Wyoming,Washakie County,56043


In [327]:
stats=stateFips.rename(columns={"Name": "state"})

In [328]:
stats=stats.drop("FIPS",axis=1)
stats

,state,Postal Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,Florida,FL
9,Georgia,GA


In [331]:
countyDF=countyDF.merge(stats, on='state', how='left')

In [336]:
countyDF=countyDF.rename(columns={"Postal Code": "abvr"})
countyDF

,state,county,id,abvr
0,Alabama,Autauga County,01001,AL
1,Alabama,Baldwin County,01003,AL
2,Alabama,Barbour County,01005,AL
3,Alabama,Bibb County,01007,AL
4,Alabama,Blount County,01009,AL
...,...,...,...,...
3136,Wyoming,Sweetwater County,56037,WY
3137,Wyoming,Teton County,56039,WY
3138,Wyoming,Uinta County,56041,WY
3139,Wyoming,Washakie County,56043,WY


In [335]:
countyDF.to_csv('county_ID_list.csv')